## Model for Saliency

###### adapted ImacheCache from nnvision and monkey_loaders from nnvision.datasets



In [ ]:
%load_ext autoreload
%autoreload 

import datajoint as dj
dj.config['enable_python_native_blobs'] = True


dj.config['database.user']= 'kanderer'
dj.config['database.password']= 'enamel-vendetta-deodorant'


schema_name = 'nnfabrik_monkey_saliency'

schema = dj.schema(schema_name, locals())
dj.config['nnfabrik.schema_name'] = schema_name


In [ ]:
import torch

import nnfabrik
from nnfabrik import builder


import numpy as np
import pickle
import os

from os import listdir
from os.path import isfile, join

import matplotlib.pyplot as plt

import nnvision

from nnvision.utility.measures import get_correlations

# NNfabrik intro: Using the builder to build the dataloader objects, models, trainer

In [ ]:
# here's where the data is on the server:
os.listdir('/data')

In [ ]:
#### loading monkey data

basepath = '/data/monkey/toliaslab/CSRF19_V4'
neuronal_data_path = os.path.join(basepath, 'neuronal_data/')
neuronal_data_files = [neuronal_data_path + f for f in listdir(neuronal_data_path) if isfile(join(neuronal_data_path, f))]
image_cache_path = os.path.join(basepath, 'images')

saliency_cache_path = os.path.join(basepath, 'images_saliency')
print(saliency_cache_path)

# Part Two: NNfabrik and DataJoint

Instead of using the builder to get the data/model/and trainer, we can use datajoint to manage that process for us.
There are Model, Dataset, and Trainer Tables. And each combination in those tables should in principle lead to a fully trained model.
For completeness, there is also a Seed table that stores the random seed, and a Fabrikant table, that stores the name and contact details of the creator (=Fabrikant).


### Make sure that a dj-database is connected. Recommended dj version is 0.12.4

In [ ]:
import datajoint as dj

dj.config['enable_python_native_blobs'] = True

from nnfabrik.templates.trained_model import TrainedModelBase
from nnfabrik.main import *
import os

if not 'stores' in dj.config:
    dj.config['stores'] = {}
    
dj.config['stores']['minio'] = {  # store in s3
    'protocol': 's3',
    'endpoint': os.environ.get('MINIO_ENDPOINT', 'DUMMY_ENDPOINT'),
    'bucket': 'nnfabrik',
    'location': 'dj-store',
    'access_key': os.environ.get('MINIO_ACCESS_KEY', 'FAKEKEY'),
    'secret_key': os.environ.get('MINIO_SECRET_KEY', 'FAKEKEY')
}

In [ ]:
a = Dataset.full_table_name

In [ ]:
Dataset.database

In [ ]:
# change this entry to reflect your datajoint username
Fabrikant().insert1(dict(fabrikant_name='kanderer',
                         email="k.anderer@t-online.de",
                         affiliation='sinzlab',
                         dj_username="kanderer"))

In [ ]:
Fabrikant()

In [ ]:
schema

In [ ]:
Seed().insert([{'seed':1000}])
Seed()

###  add entries for dataset, model, and trainer, with their corresponding configurations

#### Dataset

In [ ]:
# Specifying the dataset function: its defined in nnvision/datasets, and has to present in the __init__.py there.
# Specifying the dataset function: its defined in nnvision/datasets, and has to present in the __init__.py there.
dataset_fn = 'nnsaliency.datasets.saliency_loaders.monkey_saliency_loader'
dataset_config = {'dataset': 'CSRF_V4',
  'neuronal_data_files': ['/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3645713184967.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646146362280.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646222772876.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646321567215.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646401671909.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646657276979.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646747773987.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3646924284383.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3647439534447.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3647531807425.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3647617982355.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3648051669051.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3648221742562.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3648475457626.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3648644839726.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3649169411059.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3649252123347.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3649689088057.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3649771255234.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3650037309346.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3650459721914.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3651417046988.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3652197206019.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3652885366025.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3653061170515.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3653226873615.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3653320713405.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3653578665399.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3653924870701.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3654009687031.pickle',
   '/data/monkey/toliaslab/CSRF19_V4/neuronal_data/CSRF19_V4_3654087847388.pickle'],
  'image_cache_path': '/data/monkey/toliaslab/CSRF19_V4/images/',
     'saliency_cache_path': '/data/monkey/toliaslab/CSRF19_V4/images_saliency_scaled/',
  'crop': [(68, 0), (126, 126)],
  'subsample': 1,
  'scale': 0.4,
  'seed': 1000,
  'batch_size': 64,
  'time_bins_sum': (3, 4, 5, 6, 7, 8, 9, 10, 11),
                  'gradient': False,
                  'logarithm': True
                 }


In [ ]:
# adds the dataset_function and dataset config that we defined above to the datase table
#Dataset().add_entry(dataset_fn, dataset_config, dataset_comment='CSRF_V1')
Dataset()

#### Model

In [ ]:
model_fn = 'nnvision.models.se_core_full_gauss_readout'
model_config = {'pad_input': False,
  'gauss_type': 'isotropic',
  'gamma_readout': 0.43,
  'gamma_input': 10,
  'layers': 6,
  'depth_separable': True,
  'n_se_blocks': 0,
  'stack': -1,
  'input_kern': 15,
  'hidden_kern': 9,
  'hidden_channels': 32,
  'init_mu_range': 0.1780563002765732,
  'init_sigma': 0.5851238789405838}
Model().add_entry(model_fn, model_config, model_comment='isotropic')
Model()

In [ ]:
trainer_fn = 'nnvision.training.nnvision_trainer'
trainer_config = dict(max_iter=5, 
                      lr_decay_steps=4, 
                      tolerance=0.0005, 
                      patience=5,
                      verbose=False, 
                      lr_init=0.0045,
                      avg_loss=False,
                      device='cuda')

#Trainer().add_entry(trainer_fn, trainer_config, trainer_comment="max_iter: 5")
Trainer()

####  The TrainedModel is a template, which can be found in nnfabrik.template.py

the trained model table is taking care of model training, and stores the model state in a part table. For further analyses of the trained model, one can either overwrite the TrainedModel definition by inheriting from the Base template class, or by attaching other tables to trained model.

In [ ]:
# creating the simples TrainedModel class
print(type(TrainedModelBase))

@schema
class TrainedModel(TrainedModelBase):
    table_comment = "Trained models"
    storage = "minio"
    model_table = Model
    dataset_table = Dataset
    trainer_table = Trainer
    seed_table = Seed
    user_table = Fabrikant

In [ ]:
#dj.Diagram(schema)

as primary keys, it has the hashes of all the configurations, and it stores the score, and the output (which are defined in the respective trainer)

## Lets populate

In [ ]:
key = dict(model_hash='cec63aa4435b2a205ec02eafc0a745ee', dataset_hash='cf34e43c337d6eb562802b3e9d3c74fd', trainer_hash = 'f03a6527ab0422767da50e67e2d543ef')
TrainedModel().populate(key, reserve_jobs=True)

In [ ]:
TrainedModel()

In [ ]:
model_hash = TrainedModel().fetch1("model_hash")

In [ ]:
# now if you want to build the model again, we can use the .load_model() function of the trained model table.
# To use the load model function, the table needs to be restricted to one Entry. 
# for example: restricting with a key:
some_key = dict(model_hash=model_hash)
TrainedModel&some_key



# How to store Test Correlation

In [ ]:


@schema
class TestCorrelation(ScoringBase):
    trainedmodel_table = TrainedModel
    unit_table = MonkeyExperiment.Units
    scoring_function = staticmethod(get_correlations)
    scoring_dataset = "test"
    scoring_attribute = "test_correlation"

TestCorrelation().populate(example_key, display_progress=True)

# How to Load a Model


In [ ]:
key = dict(model_hash='07b08b47c3f1805168980920767c4e32', dataset_hash='6cdb8f03fa4d0e752d6b2eb886c83e8c', trainer_hash = 'f03a6527ab0422767da50e67e2d543ef')

dataloader, model = (TrainedModel & key).load_model()

In [ ]:
# that is the trained model, with the state dict loaded and all. lets set to eval and start using it
model.eval();

In [ ]:
from nnvision.utility.measures import get_correlations
test_correlation = get_correlations(model, dataloader["test"], device='cuda', as_dict=False, per_neuron=True)

print(test_correlation)

# Parameter Extension

In [ ]:
# There's also the parameter extension, so that you can restrict with the config objects as well.

In [ ]:
from nnfabrik.utility.dj_helpers import create_param_expansion, make_definition
ModelExpanded = create_param_expansion('nnvision.models.se_core_full_gauss_readout', Model,fn_field='model_fn', config_field='model_config')
ModelParams = schema(ModelExpanded)

In [ ]:
ModelParams()

In [ ]:
ModelParams.populate()

In [ ]:
ModelParams()

In [ ]:
# for example:
Model*ModelParams&"hidden_kern=5"

In [ ]:
# Now you can just use that for building the model:
dataloaders, model =(TrainedModel & ((Model * ModelParams)&"hidden_kern=5")).load_model()